# Data Exploration: Honolulu Airport Temperature

**Goal**: Explore daily temperature data from Honolulu International Airport, understand its structure, handle missing values, and prepare a clean dataset for the workshop.

### Data Source

Station observation data can be obtained from the **NCEI Past Weather** portal:  
🔗 https://www.ncei.noaa.gov/access/past-weather/Hawaii

We are using station **USW00022521** (Honolulu International Airport, HI).  
The CSV spans from **1939 to 2026** and contains the following columns:

| Column | Description |
|--------|-------------|
| `Date` | Observation date |
| `TAVG` | Average temperature (°C) |
| `TMAX` | Maximum temperature (°C) |
| `TMIN` | Minimum temperature (°C) |
| `PRCP` | Precipitation (inches) |
| `SNOW` | Snowfall (inches) |
| `SNWD` | Snow depth (inches) |

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "monospace"

## 1. Load & Inspect Data

In [ ]:
# The first row of the CSV is the station name header, so we skip it.
df = pd.read_csv(
    "../data/USW00022521.csv",
    skiprows=1,
    parse_dates=["Date"],
)

# Shorten the column names for convenience
df.columns = [c.split(" (")[0].strip() for c in df.columns]

# Convert Temperature Columns from Fahrenheit to Celsius
temp_cols = ["TAVG", "TMAX", "TMIN"]
for c in temp_cols:
    if c in df.columns:
        df[c] = (df[c] - 32) * 5/9

df = df.set_index("Date").sort_index()
print(f"Date range : {df.index.min().date()}  →  {df.index.max().date()}")
print(f"Total rows : {len(df):,}")
df.head()

In [ ]:
df[["TAVG", "TMAX", "TMIN"]].info()

In [ ]:
df[["TAVG", "TMAX", "TMIN"]].describe()

## 2. Visualize Raw Data & Missing Values

Plotting all three temperature columns reveals which eras have which variables available.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))

ax.plot(df.index, df["TMAX"], lw=0.3, alpha=0.6, label="TMAX", color="tomato")
ax.plot(df.index, df["TMIN"], lw=0.3, alpha=0.6, label="TMIN", color="steelblue")
ax.plot(df.index, df["TAVG"], lw=0.3, alpha=0.8, label="TAVG", color="black")

ax.set_title("Raw Temperature Data — Honolulu Airport (USW00022521)")
ax.set_ylabel("Temperature (°C)")
ax.legend(loc="upper left")
ax.set_xlim(df.index.min(), df.index.max())
fig.tight_layout()

### Missingness Heatmap

Let's create a heatmap showing the fraction of missing days per month for each temperature variable.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 9), sharex=True, sharey=True)

for ax, col in zip(axes, ["TAVG", "TMAX", "TMIN"]):
    # 1 = missing, 0 = present
    missing = df[col].isna().astype(int)
    missing_pct = missing.groupby([missing.index.year, missing.index.month]).mean()
    missing_pct.index = pd.MultiIndex.from_tuples(missing_pct.index, names=["Year", "Month"])
    pivot = missing_pct.unstack(level="Month")

    im = ax.imshow(
        pivot.values.T,
        aspect="auto",
        cmap="RdYlGn_r",
        vmin=0,
        vmax=1,
        extent=[pivot.index.min(), pivot.index.max(), 12.5, 0.5],
    )
    ax.set_ylabel("Month")
    ax.set_title(f"{col} — Missing Fraction by Year × Month")
    ax.set_yticks(range(1, 13))

axes[-1].set_xlabel("Year")
fig.colorbar(im, ax=axes, label="Fraction Missing", shrink=0.5, pad=0.015)

In [ ]:
# Summary counts
for col in ["TAVG", "TMAX", "TMIN"]:
    n_total = len(df)
    n_valid = df[col].notna().sum()
    print(f"{col}: {n_valid:,} / {n_total:,} valid  ({100*n_valid/n_total:.1f}%)  |  {n_total - n_valid:,} missing")

## 3. Data Filling — Unified Average Temperature

We will create a single, continuous `TAVG_filled` column using a **three-step strategy**:

1. **Use `TAVG`** wherever it already exists.  
2. Where `TAVG` is missing but `TMAX` and `TMIN` are present → compute `(TMAX + TMIN) / 2`.  
3. Any remaining gaps → **time-based interpolation** (`interpolate(method='time')`).

In [ ]:
# Step 1: start with TAVG
df["TAVG_filled"] = df["TAVG"].copy()

# Step 2: fill from (TMAX + TMIN) / 2
mask_missing = df["TAVG_filled"].isna()
mask_minmax = df["TMAX"].notna() & df["TMIN"].notna()
df.loc[mask_missing & mask_minmax, "TAVG_filled"] = (
    (df["TMAX"] + df["TMIN"]) / 2
)

print("After step 2 (TMAX/TMIN average):")
print(f"  Remaining NaN: {df['TAVG_filled'].isna().sum():,}")

# Step 3: interpolate any remaining gaps
df["TAVG_filled"] = df["TAVG_filled"].interpolate(method="time")

print("\nAfter step 3 (time interpolation):")
print(f"  Remaining NaN: {df['TAVG_filled'].isna().sum():,}")

### Validate the filling

Plot original `TAVG` alongside the filled version so we can visually verify that the filling is sensible.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))

ax.plot(df.index, df["TAVG_filled"], lw=0.4, alpha=0.8, label="TAVG (filled)", color="darkorange")
ax.plot(df.index, df["TAVG"], lw=0.4, alpha=0.5, label="Original TAVG", color="gray")

ax.set_title("Original vs Filled Average Temperature")
ax.set_ylabel("Temperature (°C)")
ax.legend()
ax.set_xlim(df.index.min(), df.index.max())
fig.tight_layout()

## 4. Temperature Analysis & Plotting

### Daily temperature with 30-day rolling average

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.plot(df.index, df["TAVG_filled"], lw=0.2, alpha=0.4, color="steelblue", label="Daily")
rolling = df["TAVG_filled"].rolling(30, center=True).mean()
ax.plot(df.index, rolling, lw=1.2, color="darkred", label="30-day rolling mean")

ax.set_title("Daily Average Temperature — Honolulu Airport (1939–2026)")
ax.set_ylabel("Temperature (°C)")
ax.legend()
ax.set_xlim(df.index.min(), df.index.max())
fig.tight_layout()

### Monthly Climatology (mean ± std)

In [ ]:
monthly_clim = df["TAVG_filled"].groupby(df.index.month).agg(["mean", "std"])
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(range(1, 13), monthly_clim["mean"], yerr=monthly_clim["std"],
       color="coral", edgecolor="black", capsize=4, alpha=0.85)
ax.set_xticks(range(1, 13))
ax.set_xticklabels(month_names)
ax.set_ylabel("Temperature (°C)")
ax.set_title("Monthly Temperature Climatology — Honolulu Airport")
ax.set_ylim(15, 35)
fig.tight_layout()

### Annual Mean Temperature Trend

In [ ]:
annual_mean = df["TAVG_filled"].resample("YS").mean()[:-1]

# Linear fit
x_years = annual_mean.index.year.values.astype(float)
y_vals = annual_mean.values
mask_valid = ~np.isnan(y_vals)
coeffs = np.polyfit(x_years[mask_valid], y_vals[mask_valid], 1)
trend_line = np.polyval(coeffs, x_years)

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(annual_mean.index, annual_mean, "o-", ms=3, color="steelblue", label="Annual Mean")
ax.plot(annual_mean.index, trend_line, "--", color="red",
        label=f"Trend: {coeffs[0]:+.3f} °C/year")
ax.set_title("Annual Mean Temperature — Honolulu Airport")
ax.set_ylabel("Temperature (°C)")
ax.legend()
plt.tight_layout()

## 5. Export Processed Data

Save the cleaned DataFrame with the filled `TAVG_filled` column for use in subsequent notebooks.

In [ ]:
import os
os.makedirs("../data/processed", exist_ok=True)

df.to_csv("../data/processed/honolulu_airport_daily_temperature.csv")